In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import math
import geopy.distance
from tqdm.notebook import tqdm
%matplotlib inline

In [72]:
hub_covid_violations_df = pd.read_csv('../data/hub_covid_violations.csv')

In [73]:
hub_covid_violations_df.head()

,Request #,Status,Date / Time Opened,Date / Time Closed,Contact Type,State Issue,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,346122,Closed,2020-04-15 11:33:26,2020-04-16 14:55:01,NaN,False,False,928 6th Ave S,NASHVILLE,17.0,37203.0,36.149040,-86.772273,"(36.1490396, -86.77227289999999)"
1,339374,Closed,2020-04-04 18:13:59,2020-04-09 13:07:22,NaN,False,False,500 Gallatin Ave,NASHVILLE,5.0,37206.0,36.181548,-86.749695,"(36.181548, -86.749695)"
2,343513,Closed,2020-04-10 10:40:08,2020-04-14 19:39:09,NaN,False,False,5720 Crossings Blvd,NaN,NaN,NaN,36.039608,-86.646541,"(36.039608, -86.646541)"
3,442996,Closed,2020-08-12 02:05:29,2020-08-12 16:03:35,NaN,False,False,7689 Hwy 70 S,NASHVILLE,22.0,37221.0,36.078933,-86.953400,"(36.07893290000001, -86.9533998)"
4,339423,Closed,2020-04-04 20:12:27,2020-04-09 14:21:09,NaN,False,False,5310 Mt View Rd,ANTIOCH,32.0,37013.0,36.048182,-86.648447,"(36.0481819, -86.6484474)"


In [74]:
hub_covid_violations_df['Longitude'] = round(hub_covid_violations_df['Longitude'], 3)
hub_covid_violations_df['Latitude'] = round(hub_covid_violations_df['Latitude'], 3)

In [75]:
clusters = pd.read_csv('../data/clusters.csv')

In [76]:
clusters

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.125891,-86.822863
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.8467716
2,Religious Retreat,Social Gathering,3/25/2020,18,NaN,NaN
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.8195333
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.7567485
...,...,...,...,...,...,...
57,One Stone Church Service,Church,9/22/2020,10,36.189960,-86.7673379
58,Miss Kelli's,Bar,10/1/2020,14,36.164369,-86.7807037
59,Nashville Rescue Mission - Women's Campus,Congregate Living,10/1/2020,70,36.167693,-86.7957557
60,Link Systems Electric,Other,10/16/2020,12,36.092924,-86.7438727


In [77]:
clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cluster Name        62 non-null     object 
 1   Facility Type       62 non-null     object 
 2   Cluster Start Date  62 non-null     object 
 3   # Cases             62 non-null     int64  
 4   Latitude            52 non-null     float64
 5   Longitude           52 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 3.0+ KB


In [78]:
clusters['Cluster Name'].value_counts().sort_values(ascending=False)

CDM Jail                                  2
Event at Clementine Hall                  1
Life Care Center Old Hickory Village      1
Standing Tall Music City                  1
NHC Place at the Trace NH                 1
                                         ..
Link Systems Electric                     1
Vanderbilt Parties                        1
Creekside Center for Rehab and Healing    1
High/Middle-School Teen Party             1
Wedding at Farm (Out of County)           1
Name: Cluster Name, Length: 61, dtype: int64

In [79]:
clusters['Longitude'] = clusters['Longitude'].str.lstrip(r'\,*')

In [80]:
clusters['Longitude'] = clusters['Longitude'].astype(float)

In [81]:
clusters['Longitude'] = round(clusters['Longitude'], 3)
clusters['Latitude'] = round(clusters['Latitude'], 3)

In [82]:
clusters_by_type = pd.read_csv('../data/clusters_by_type.csv')

In [83]:
clusters_by_type.head()

,Cluster Type,Number of Clusters
0,Bar,7
1,Church,2
2,College / University,5
3,Commercial-Warehouse,12
4,Congregate Living,13


In [84]:
clusters_violations = clusters[clusters['Latitude'].notna()].merge(
    hub_covid_violations_df, 
    on=['Latitude', 'Longitude'], 
    how='inner'
)

In [85]:
clusters_violations

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude,Request #,Status,Date / Time Opened,Date / Time Closed,Contact Type,State Issue,Closed When Created,Address,City,Council District,ZIP,Mapped Location
0,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152,-86.847,407619,Closed,2020-07-04 16:47:23,2020-07-06 17:42:16,NaN,False,False,5001 Charlotte Ave,NASHVILLE,24.0,37209.0,"(36.1518346, -86.8474861)"
1,Rescue Mission-Fairgrounds,Congregate Living,4/30/2020,156,36.145,-86.765,360354,Closed,2020-05-06 14:31:55,2020-05-07 15:10:14,NaN,False,False,1212 3rd Ave S,NASHVILLE,17.0,37210.0,"(36.14462, -86.76486)"
2,The Opal at Music City,LTCF,6/3/2020,27,36.162,-86.663,388854,Closed,2020-06-10 19:27:11,2020-06-11 14:46:48,NaN,False,False,211 Donelson Pike,NASHVILLE,15.0,37214.0,"(36.1619421, -86.66331210000001)"
3,The Opal at Music City,LTCF,6/3/2020,27,36.162,-86.663,417578,Closed,2020-07-15 22:26:52,2020-07-16 17:05:01,NaN,False,False,211 Donelson Pike,NASHVILLE,15.0,37214.0,"(36.1619254, -86.6633155)"
4,The Opal at Music City,LTCF,6/3/2020,27,36.162,-86.663,431369,Closed,2020-07-29 20:02:51,2020-07-30 18:07:05,NaN,False,False,211 Donelson Pike,NASHVILLE,15.0,37214.0,"(36.1619254, -86.6633155)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Loser's,Bar,6/26/2020,29,36.161,-86.784,420551,Closed,2020-07-19 23:47:16,2020-07-20 17:42:45,NaN,False,False,167 Rosa L Parks Blvd,NASHVILLE,19.0,37203.0,"(36.1605824, -86.7838543)"
65,Loser's,Bar,6/26/2020,29,36.161,-86.784,421054,Closed,2020-07-20 16:37:45,2020-07-21 17:24:48,NaN,False,False,167 Rosa L Parks Blvd,NASHVILLE,19.0,37203.0,"(36.1605824, -86.7838543)"
66,Loser's,Bar,6/26/2020,29,36.161,-86.784,440666,Closed,2020-08-07 21:46:09,2020-08-10 15:58:58,NaN,False,False,167 Rosa L Parks Blvd,NASHVILLE,19.0,37203.0,"(36.16076381160593, -86.78359290674591)"
67,Loser's,Bar,6/26/2020,29,36.161,-86.784,449441,Closed,2020-08-20 23:14:53,2020-08-21 14:04:38,NaN,False,False,167 Rosa L Parks Blvd,NASHVILLE,19.0,37203.0,"(36.1605824, -86.7838543)"


In [86]:
clusters_violations['Cluster Name'].value_counts()

Kid Rock's Big Ass Honky Tonk    52
Dogwood                           5
Loser's                           4
The Opal at Music City            3
Event at Clementine Hall          1
Miss Kelli's                      1
Rescue Mission-Fairgrounds        1
Dawghouse Saloon                  1
Lipscomb Elam Hall job site       1
Name: Cluster Name, dtype: int64

In [87]:
clusters_violations[clusters_violations['Cluster Name'] == "Kid Rock's Big Ass Honky Tonk"]

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude,Request #,Status,Date / Time Opened,Date / Time Closed,Contact Type,State Issue,Closed When Created,Address,City,Council District,ZIP,Mapped Location
12,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,374898,Closed,2020-05-22 17:18:14,2020-05-28 14:53:55,NaN,False,False,420 Broadway,NASHVILLE,19.0,37203.0,"(36.160817086423826, -86.77826539600221)"
13,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,367424,Closed,2020-05-14 14:53:12,2020-05-15 19:39:24,NaN,False,False,412 Broadway,NASHVILLE,19.0,37203.0,"(36.1609069, -86.77782789999999)"
14,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,371706,Closed,2020-05-19 15:49:07,2020-05-20 15:40:34,NaN,False,False,428 Broadway,NASHVILLE,19.0,37203.0,"(36.1607991, -86.7784958)"
15,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,426950,Closed,2020-07-26 14:27:51,2020-07-27 20:13:20,NaN,False,False,422 Broadway,NASHVILLE,19.0,37203.0,"(36.1607689, -86.77821)"
16,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,369557,Closed,2020-05-15 21:03:07,2020-05-18 18:25:08,NaN,False,False,412 Broadway,NASHVILLE,19.0,37203.0,"(36.1609069, -86.77782789999999)"
17,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,360510,Closed,2020-05-06 15:57:32,2020-05-07 15:08:02,NaN,False,False,422 Broadway,NASHVILLE,19.0,37203.0,"(36.1607689, -86.77821)"
18,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,369786,Closed,2020-05-16 04:06:41,2020-05-18 18:24:49,NaN,False,False,412 Broadway,NASHVILLE,19.0,37203.0,"(36.1609069, -86.77782789999999)"
19,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,369760,Closed,2020-05-16 01:43:28,2020-05-18 18:24:58,NaN,False,False,412 Broadway,NASHVILLE,19.0,37203.0,"(36.1609069, -86.77782789999999)"
20,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,402503,Closed,2020-06-28 15:41:45,2020-06-29 19:05:19,NaN,False,False,412 Broadway,NASHVILLE,19.0,37203.0,"(36.160906, -86.777846)"
21,Kid Rock's Big Ass Honky Tonk,Bar,6/26/2020,15,36.161,-86.778,369871,Closed,2020-05-16 16:38:51,2020-05-18 18:24:37,NaN,False,False,412 Broadway,NASHVILLE,19.0,37203.0,"(36.1609069, -86.77782789999999)"


## Look at some Folium Maps

## Starting with just looking at all the clusters

COVID Icon - Icons made by <a href="https://www.flaticon.com/authors/freepik" title="Freepik">Freepik</a> from <a href="https://www.flaticon.com/" title="Flaticon"> www.flaticon.com</a>

In [88]:
center = [36.1672, -86.7816]
nashville_cluster_map = folium.Map(location = center, zoom_start=11)

In [90]:
for row_index, row_values in clusters[clusters['Latitude'].notna()].iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Cluster Name'])
    icon = folium.features.CustomIcon('covid4.png')
    marker = folium.Marker(
        icon = icon,
        location = loc,
        popup = folium.Popup(pop,
                     min_width=200,
                     max_width=500))
    
    marker.add_to(nashville_cluster_map)
    
nashville_cluster_map

In [68]:
center = [36.1672, -86.7816]
nashville_covid_map = folium.Map(location = center, zoom_start=12)

In [69]:
for row_index, row_values in clusters_violations.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Cluster Name'])
    marker = folium.CircleMarker(
        location = loc,
        popup = pop)
    
    marker.add_to(nashville_covid_map)

In [70]:
nashville_covid_map

Loser's looks like it's located in the wrong spot - so it may not be correct for a cluster??